# Xây dựng model

In [37]:
from lib_all import *

## I. Module VGG

In [38]:
def create_vgg():
    layers = []

    input_channels = 3
    configs = [64, 64, 'M',
               128, 128, 'M',
               256, 256, 256, 'MC',
               512, 512, 512, 'M',
               512, 512, 512]
    relu = nn.ReLU(inplace = True) #inplace la khong muon luu ket qua tren memory => Tiet kiem bo nho

    for config in configs:
        if config == 'M': #lam tron bang cach lay nguyen phan nguyen
            layers += [nn.MaxPool2d(kernel_size = 2, stride = 2)] # 2x2 va lay ra pixel lon nhat trong 4 pixel do => giam di 1 nua, stride la dich di 2 pixel/ lan
        elif config == 'MC': # lam tron len
            layers += [nn.MaxPool2d(kernel_size = 2, stride = 2, ceil_mode = True)]
        else:
            conv2d = nn.Conv2d(input_channels, config, kernel_size = 3, padding  =1) # config la so channels ouput, padding la 1
            layers += [conv2d, relu]

            input_channels = config

    pool5  = nn.MaxPool2d(kernel_size = 3, stride = 1, padding  = 1)
    layers += [pool5]
    conv6 = nn.Conv2d(input_channels,1024,  kernel_size = 3, padding = 6, dilation = 6) # khoang cach giua cac phan tu trong kernel
    layers += [conv6, relu]
    conv7 = nn.Conv2d(1024,1024, kernel_size = 1)
    layers += [conv7, relu]

    return nn.ModuleList(layers)

## II. Module Extras

In [39]:
def create_extras():
    layers = []
    configs = [256, 512, 128, 256, 128, 256, 128, 256 ]

    # block 1
    input_channels = 1024
    output_channels = configs[0]
    conv1 = nn.Conv2d(input_channels, output_channels, kernel_size=1)
    layers+= [conv1]

    # block 2
    input_channels = configs[0]
    output_channels = configs[1]
    conv2 = nn.Conv2d(input_channels, output_channels, kernel_size=3, stride= 2, padding= 1)
    layers+= [conv2]

    # block 3
    input_channels = configs[1]
    output_channels = configs[2]
    conv3 = nn.Conv2d(input_channels, output_channels, kernel_size=1)
    layers+= [conv3]

    # block 4
    input_channels = configs[2]
    output_channels = configs[3]
    conv4 = nn.Conv2d(input_channels, output_channels, kernel_size=3, stride= 2, padding= 1)
    layers+= [conv4]

    # block 5
    input_channels = configs[3]
    output_channels = configs[4]
    conv5 = nn.Conv2d(input_channels, output_channels, kernel_size=1)
    layers+= [conv5]

    # block 6
    input_channels = configs[4]
    output_channels = configs[5]
    conv6 = nn.Conv2d(input_channels, output_channels, kernel_size=3)
    layers+= [conv6]

    # block 7
    input_channels = configs[5]
    output_channels = configs[6]
    conv7 = nn.Conv2d(input_channels, output_channels, kernel_size=1)
    layers+= [conv7]

    # block 8
    input_channels = configs[6]
    output_channels = configs[7]
    conv8 = nn.Conv2d(input_channels, output_channels, kernel_size=1)
    layers+= [conv8]

    return nn.ModuleList(layers)

## III.Module loc

bbox_ratio_num: tỉ lệ để nhân với số lượng bbox ra 8732

In [40]:
def create_loc():
    list_input_channels = [512, 1024,512, 256, 256,256]
    list_bbox_weigth = [4,6,6,6,4,4]
    num_offset = 4
    layers = []

    for index_source in range(0, 6):
        input_channels = list_input_channels[index_source]
        output_channels = list_bbox_weigth[index_source]*num_offset
        conv = nn.Conv2d(input_channels,output_channels, kernel_size=3, padding=1)
        layers += [conv]

    return nn.ModuleList(layers)

## IV. Module conf

num_classes: truyen vao 21

In [41]:
def create_conf(num_classes = 21):
    list_input_channels = [512, 1024,512, 256, 256,256]
    list_bbox_weigth = [4,6,6,6,4,4]
    layers = []

    for index_source in range(0, 6):
        input_channels = list_input_channels[index_source]
        output_channels = list_bbox_weigth[index_source]*num_classes
        conv = nn.Conv2d(input_channels,output_channels, kernel_size=3, padding=1)
        layers += [conv]

    return nn.ModuleList(layers)

# Test

Kiểm tra module vgg

In [42]:
vgg = create_vgg()
print(vgg)

ModuleList(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
  (17): Conv2d(256, 512, kernel_siz

Kiểm tra module extras

In [43]:
extras = create_extras()
print (extras)

ModuleList(
  (0): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
  (1): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (2): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1))
  (3): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (4): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
  (5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
  (6): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
  (7): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1))
)


Kiểm tra module loc

In [44]:
loc = create_loc()
print (loc)

ModuleList(
  (0): Conv2d(512, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): Conv2d(1024, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): Conv2d(512, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): Conv2d(256, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): Conv2d(256, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): Conv2d(256, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)


Kiểm tra module conf

In [45]:
conf = create_conf()
print(conf)

ModuleList(
  (0): Conv2d(512, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): Conv2d(1024, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): Conv2d(512, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): Conv2d(256, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): Conv2d(256, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): Conv2d(256, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)
